In [1]:
!pip install linearmodels

In [3]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import statsmodels.formula.api as smf
# from linearmodels.panel import PanelOLS
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [5]:
from linearmodels.panel import PanelOLS

## Подгрузим и смерджим все масссивы

In [6]:
df_fin = pd.read_csv('/Users/rusya/Documents/synthetic_data/dataframe_for_analysis/all_data_right_format.csv')

/var/folders/v_/zqcyjtwn0d70thqjm302x3zh0000gn/T/ipykernel_38843/67941192.py:1: DtypeWarning: Columns (37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fin = pd.read_csv('/Users/rusya/Documents/synthetic_data/dataframe_for_analysis/all_data_right_format.csv')


## Регрессия для среднего ответа по стране и страновых показателей

In [7]:
df_fin

,Unnamed: 0,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,...,Q195_yandex,GDP_per_capita_2024,ict_idx,GDP_wvs_year,rd_idx_gpd,TradAgg,SurvSAgg,CPI_idx,HDI_idx,is_genderless
0,0,2018,AND,Andorra,3,2,1,2,3,1270,...,3.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
1,1,2018,AND,Andorra,2,1,3,3,2,1270,...,2.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
2,2,2018,AND,Andorra,2,1,3,2,3,810,...,2.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
3,3,2018,AND,Andorra,3,2,1,2,4,1270,...,3.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
4,4,2018,AND,Andorra,2,1,1,2,4,1270,...,3.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85303,85303,2020,ZWE,Zimbabwe,2,2,0,2,3,3830,...,2.0,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0
85304,85304,2020,ZWE,Zimbabwe,2,1,3,2,2,3200,...,3.0,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0
85305,85305,2020,ZWE,Zimbabwe,1,2,1,2,4,3830,...,3.0,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0
85306,85306,2020,ZWE,Zimbabwe,2,2,1,1,4,3830,...,3.0,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0


In [9]:
# Список нежелательных кодов
bad_codes = [-2, 3900, -1, -5, -4, 2190]

# Отфильтровать датафрейм, оставив только строки, где Q272 не в bad_codes
df_fin = df_fin[~df_fin['Q272'].isin(bad_codes)]

df_fin

,Unnamed: 0,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,...,Q195_yandex,GDP_per_capita_2024,ict_idx,GDP_wvs_year,rd_idx_gpd,TradAgg,SurvSAgg,CPI_idx,HDI_idx,is_genderless
0,0,2018,AND,Andorra,3,2,1,2,3,1270,...,3.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
1,1,2018,AND,Andorra,2,1,3,3,2,1270,...,2.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
2,2,2018,AND,Andorra,2,1,3,2,3,810,...,2.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
3,3,2018,AND,Andorra,3,2,1,2,4,1270,...,3.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
4,4,2018,AND,Andorra,2,1,1,2,4,1270,...,3.0,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85303,85303,2020,ZWE,Zimbabwe,2,2,0,2,3,3830,...,2.0,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0
85304,85304,2020,ZWE,Zimbabwe,2,1,3,2,2,3200,...,3.0,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0
85305,85305,2020,ZWE,Zimbabwe,1,2,1,2,4,3830,...,3.0,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0
85306,85306,2020,ZWE,Zimbabwe,2,2,1,1,4,3830,...,3.0,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0


In [12]:
# df — ваш исходный DataFrame с колонками 'country' и 'lang'
# 1) Считаем количество каждой пары (country, lang)
df_counts = (
    df_fin
    .groupby(['country', 'Q272'])
    .size()
    .reset_index(name='count')
)

# 2) Для каждой страны выбираем строку с максимальным count
idx = df_counts.groupby('country')['count'].idxmax()
result_df = df_counts.loc[idx, ['country', 'Q272']].reset_index(drop=True)

result_df

,country,Q272
0,Andorra,1270
1,Argentina,1270
2,Armenia,200
3,Bangladash,490
4,Bolivia,1270
...,...,...
57,Uruguay,1270
58,Uzbekistan,4450
59,Venezuela,1270
60,Vietnam,4470


In [13]:
result_df.rename(columns={'Q272': "Q272_most_common"}, inplace=True)

In [14]:
result_df

,country,Q272_most_common
0,Andorra,1270
1,Argentina,1270
2,Armenia,200
3,Bangladash,490
4,Bolivia,1270
...,...,...
57,Uruguay,1270
58,Uzbekistan,4450
59,Venezuela,1270
60,Vietnam,4470


In [21]:
result_df.to_csv('/Users/rusya/Documents/synthetic_data/dataframe_for_analysis/Q272_most_common.csv')

In [15]:
df_fin = df_fin.merge(
    result_df[[
        'country',
        'Q272_most_common'
    ]],
    on='country',
    how='left'
)

In [16]:
df_fin

,Unnamed: 0,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,...,GDP_per_capita_2024,ict_idx,GDP_wvs_year,rd_idx_gpd,TradAgg,SurvSAgg,CPI_idx,HDI_idx,is_genderless,Q272_most_common
0,0,2018,AND,Andorra,3,2,1,2,3,1270,...,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270
1,1,2018,AND,Andorra,2,1,3,3,2,1270,...,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270
2,2,2018,AND,Andorra,2,1,3,2,3,810,...,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270
3,3,2018,AND,Andorra,3,2,1,2,4,1270,...,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270
4,4,2018,AND,Andorra,2,1,1,2,4,1270,...,46359.0,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82937,85303,2020,ZWE,Zimbabwe,2,2,0,2,3,3830,...,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0,3830
82938,85304,2020,ZWE,Zimbabwe,2,1,3,2,2,3200,...,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0,3830
82939,85305,2020,ZWE,Zimbabwe,1,2,1,2,4,3830,...,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0,3830
82940,85306,2020,ZWE,Zimbabwe,2,2,1,1,4,3830,...,2073.0,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0,3830


In [20]:
df_fin.to_csv('/Users/rusya/Documents/synthetic_data/dataframe_for_analysis/all_ans_most_common_lang_for_each_country.csv')

In [22]:
df_fsi = pd.read_csv('/Users/rusya/Documents/synthetic_data/dataframe_for_analysis/Q272_most_common_with_fsi.csv')
df_fsi

,Unnamed: 0,country,Q272_most_common,fsi
0,0,Andorra,1270,1.0
1,1,Argentina,1270,1.0
2,2,Armenia,200,3.0
3,3,Bangladash,490,3.0
4,4,Bolivia,1270,1.0
...,...,...,...,...
57,57,Uruguay,1270,1.0
58,58,Uzbekistan,4450,3.0
59,59,Venezuela,1270,1.0
60,60,Vietnam,4470,3.0


In [23]:
df_fin = df_fin.merge(
    df_fsi[[
        'country',
        'fsi'
    ]],
    on='country',
    how='left'
)

df_fin

,Unnamed: 0,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,...,ict_idx,GDP_wvs_year,rd_idx_gpd,TradAgg,SurvSAgg,CPI_idx,HDI_idx,is_genderless,Q272_most_common,fsi
0,0,2018,AND,Andorra,3,2,1,2,3,1270,...,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270,1.0
1,1,2018,AND,Andorra,2,1,3,3,2,1270,...,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270,1.0
2,2,2018,AND,Andorra,2,1,3,2,3,810,...,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270,1.0
3,3,2018,AND,Andorra,3,2,1,2,4,1270,...,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270,1.0
4,4,2018,AND,Andorra,2,1,1,2,4,1270,...,88.8,42819.774580,NaN,0.5486,2.1823,NaN,0.873,1.0,1270,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82937,85303,2020,ZWE,Zimbabwe,2,2,0,2,3,3830,...,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0,3830,2.0
82938,85304,2020,ZWE,Zimbabwe,2,1,3,2,2,3200,...,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0,3830,2.0
82939,85305,2020,ZWE,Zimbabwe,1,2,1,2,4,3830,...,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0,3830,2.0
82940,85306,2020,ZWE,Zimbabwe,2,2,1,1,4,3830,...,47.7,1730.413489,0.21,-0.5850,-1.8709,24.0,0.584,0.0,3830,2.0


In [24]:
df_fin.columns

Index(['Unnamed: 0', 'A_YEAR', 'code', 'country', 'X003R2', 'Q260', 'Q275',
       'Q288R', 'Q287', 'Q272', 'Q273', 'Q274', 'Q289', 'Q158', 'Q159', 'Q160',
       'Q161', 'Q162', 'Q177', 'Q178', 'Q179', 'Q180', 'Q181', 'Q182', 'Q183',
       'Q184', 'Q185', 'Q186', 'Q187', 'Q188', 'Q189', 'Q190', 'Q191', 'Q192',
       'Q193', 'Q194', 'Q195', 'Q158_gpt', 'Q159_gpt', 'Q160_gpt', 'Q161_gpt',
       'Q162_gpt', 'Q177_gpt', 'Q178_gpt', 'Q179_gpt', 'Q180_gpt', 'Q181_gpt',
       'Q182_gpt', 'Q183_gpt', 'Q184_gpt', 'Q185_gpt', 'Q186_gpt', 'Q187_gpt',
       'Q188_gpt', 'Q189_gpt', 'Q190_gpt', 'Q191_gpt', 'Q192_gpt', 'Q193_gpt',
       'Q194_gpt', 'Q195_gpt', 'Q158_yandex', 'Q159_yandex', 'Q160_yandex',
       'Q161_yandex', 'Q162_yandex', 'Q183_yandex', 'Q184_yandex',
       'Q185_yandex', 'Q186_yandex', 'Q187_yandex', 'Q188_yandex',
       'Q189_yandex', 'Q190_yandex', 'Q191_yandex', 'Q192_yandex',
       'Q193_yandex', 'Q194_yandex', 'Q195_yandex', 'GDP_per_capita_2024',
       'ict_idx', 

In [ ]:
df_fin.to_csv('/Users/rusya/Documents/synthetic_data/dataframe_for_analysis/all_data_q272_most_common_plus_fsi')

In [8]:
df_fin[['GDP_wvs_year', 'TradAgg', 'SurvSAgg', 'rd_idx_gpd', 'GDP_per_capita_2024', 'ict_idx', 'TradAgg', 'SurvSAgg', 'CPI_idx', 'HDI_idx']].corr()

,GDP_wvs_year,TradAgg,SurvSAgg,rd_idx_gpd,GDP_per_capita_2024,ict_idx,TradAgg,SurvSAgg,CPI_idx,HDI_idx
GDP_wvs_year,1.000000,0.615605,0.756788,0.659007,0.977045,0.566155,0.615605,0.756788,0.895535,0.791179
TradAgg,0.615605,1.000000,0.574217,0.724083,0.562720,0.557917,1.000000,0.574217,0.667786,0.758474
SurvSAgg,0.756788,0.574217,1.000000,0.489146,0.694311,0.374066,0.574217,1.000000,0.741467,0.675969
rd_idx_gpd,0.659007,0.724083,0.489146,1.000000,0.646866,0.533444,0.724083,0.489146,0.612835,0.691714
GDP_per_capita_2024,0.977045,0.562720,0.694311,0.646866,1.000000,0.574064,0.562720,0.694311,0.856228,0.770135
ict_idx,0.566155,0.557917,0.374066,0.533444,0.574064,1.000000,0.557917,0.374066,0.558148,0.827913
TradAgg,0.615605,1.000000,0.574217,0.724083,0.562720,0.557917,1.000000,0.574217,0.667786,0.758474
SurvSAgg,0.756788,0.574217,1.000000,0.489146,0.694311,0.374066,0.574217,1.000000,0.741467,0.675969
CPI_idx,0.895535,0.667786,0.741467,0.612835,0.856228,0.558148,0.667786,0.741467,1.000000,0.752321
HDI_idx,0.791179,0.758474,0.675969,0.691714,0.770135,0.827913,0.758474,0.675969,0.752321,1.000000


Высокая корреляция между: 

GDP_per_capita_2024	- GDP_wvs_year

CPI_idx - GDP_wvs_year

HDI_idx - GDP_wvs_year

SurvSAgg - GDP_wvs_year

HDI_idx - TradAgg

и тд

### Q182 - Homosexuality

In [444]:
df_lcl = df_fin[
    df_fin["Q184"].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
].copy()

df_lcl.drop(columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'], inplace=True)

df_lcl.dropna(subset=['Q184', 'Q184_gpt', 'TradAgg', 'GDP_wvs_year', 'SurvSAgg', 'HDI_idx'], inplace=True)

In [445]:
df_lcl = df_lcl.groupby(['country', 'code', 'A_YEAR'])[['Q184', 'Q184_gpt', 'Q184_yandex', 'TradAgg', 'GDP_wvs_year', 'SurvSAgg', 'HDI_idx']].mean().reset_index()

In [446]:
model_real = smf.ols('Q184 ~ TradAgg + SurvSAgg + HDI_idx', data = df_lcl)

result_real = model_real.fit(cov_types='H3')
print(result_real.summary())

model_gpt = smf.ols('Q184_gpt ~ TradAgg + SurvSAgg + HDI_idx', data = df_lcl)

result_gpt = model_gpt.fit(cov_types='H3')
print(result_gpt.summary())

model_yandex = smf.ols('Q184_yandex ~ TradAgg + SurvSAgg + HDI_idx', data = df_lcl)

result_yandex = model_yandex.fit(cov_types='H3')
print(result_yandex.summary())

                            OLS Regression Results                            
Dep. Variable:                   Q184   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     96.82
Date:                Wed, 11 Jun 2025   Prob (F-statistic):           7.89e-21
Time:                        00:17:57   Log-Likelihood:                -46.096
No. Observations:                  54   AIC:                             100.2
Df Residuals:                      50   BIC:                             108.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2871      0.959      3.427      0.0

### Q160 - We depend too much on science and not enough on faith

In [374]:
df_lcl = df_fin[
    df_fin["Q160"].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) &
    df_fin["Q160_gpt"].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
].copy()

df_lcl['Q160_gpt'] = df_lcl['Q160_gpt'].astype(int)

df_lcl.drop(columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'], inplace=True)
df_lcl.dropna(inplace=True)

df_lcl = df_lcl.groupby(['country', 'code', 'A_YEAR'])[['Q160', 'Q160_gpt', 'TradAgg', 'GDP_wvs_year', 'ict_idx', 'HDI_idx', 'rd_idx_gpd']].mean().reset_index()

In [375]:
df_fin[['GDP_wvs_year', 'TradAgg', 'SurvSAgg', 'rd_idx_gpd', 'GDP_per_capita_2024', 'ict_idx', 'TradAgg', 'SurvSAgg', 'CPI_idx', 'HDI_idx']].corr()

,GDP_wvs_year,TradAgg,SurvSAgg,rd_idx_gpd,GDP_per_capita_2024,ict_idx,TradAgg,SurvSAgg,CPI_idx,HDI_idx
GDP_wvs_year,1.000000,0.615605,0.756788,0.659007,0.977045,0.566155,0.615605,0.756788,0.895535,0.791179
TradAgg,0.615605,1.000000,0.574217,0.724083,0.562720,0.557917,1.000000,0.574217,0.667786,0.758474
SurvSAgg,0.756788,0.574217,1.000000,0.489146,0.694311,0.374066,0.574217,1.000000,0.741467,0.675969
rd_idx_gpd,0.659007,0.724083,0.489146,1.000000,0.646866,0.533444,0.724083,0.489146,0.612835,0.691714
GDP_per_capita_2024,0.977045,0.562720,0.694311,0.646866,1.000000,0.574064,0.562720,0.694311,0.856228,0.770135
ict_idx,0.566155,0.557917,0.374066,0.533444,0.574064,1.000000,0.557917,0.374066,0.558148,0.827913
TradAgg,0.615605,1.000000,0.574217,0.724083,0.562720,0.557917,1.000000,0.574217,0.667786,0.758474
SurvSAgg,0.756788,0.574217,1.000000,0.489146,0.694311,0.374066,0.574217,1.000000,0.741467,0.675969
CPI_idx,0.895535,0.667786,0.741467,0.612835,0.856228,0.558148,0.667786,0.741467,1.000000,0.752321
HDI_idx,0.791179,0.758474,0.675969,0.691714,0.770135,0.827913,0.758474,0.675969,0.752321,1.000000


In [376]:
model_real = smf.ols('Q160 ~ ict_idx + rd_idx_gpd + GDP_wvs_year', data = df_lcl)

result_real = model_real.fit(cov_types='H3')
print(result_real.summary())

model_gpt = smf.ols('Q160_gpt ~ ict_idx + rd_idx_gpd + GDP_wvs_year', data = df_lcl)

result_gpt = model_gpt.fit(cov_types='H3')
print(result_gpt.summary())

                            OLS Regression Results                            
Dep. Variable:                   Q160   R-squared:                       0.305
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     6.576
Date:                Tue, 10 Jun 2025   Prob (F-statistic):           0.000881
Time:                        17:55:27   Log-Likelihood:                -61.046
No. Observations:                  49   AIC:                             130.1
Df Residuals:                      45   BIC:                             137.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        5.3077      0.747      7.105   

## Регрессия fixed-effects с демографией

In [377]:
df_fin.columns

Index(['A_YEAR', 'code', 'country', 'X003R2', 'Q260', 'Q275', 'Q288R', 'Q287',
       'Q272', 'Q273', 'Q274', 'Q289', 'Q158', 'Q159', 'Q160', 'Q161', 'Q162',
       'Q177', 'Q178', 'Q179', 'Q180', 'Q181', 'Q182', 'Q183', 'Q184', 'Q185',
       'Q186', 'Q187', 'Q188', 'Q189', 'Q190', 'Q191', 'Q192', 'Q193', 'Q194',
       'Q195', 'Q158_gpt', 'Q159_gpt', 'Q160_gpt', 'Q161_gpt', 'Q162_gpt',
       'Q177_gpt', 'Q178_gpt', 'Q179_gpt', 'Q180_gpt', 'Q181_gpt', 'Q182_gpt',
       'Q183_gpt', 'Q184_gpt', 'Q185_gpt', 'Q186_gpt', 'Q187_gpt', 'Q188_gpt',
       'Q189_gpt', 'Q190_gpt', 'Q191_gpt', 'Q192_gpt', 'Q193_gpt', 'Q194_gpt',
       'Q195_gpt', 'Q158_yandex', 'Q159_yandex', 'Q160_yandex', 'Q161_yandex',
       'Q162_yandex', 'Q183_yandex', 'Q184_yandex', 'Q185_yandex',
       'Q186_yandex', 'Q187_yandex', 'Q188_yandex', 'Q189_yandex',
       'Q190_yandex', 'Q191_yandex', 'Q192_yandex', 'Q193_yandex',
       'Q194_yandex', 'Q195_yandex', 'GDP_per_capita_2024', 'ict_idx',
       'GDP_wvs_year'

In [378]:
print(df_fin.loc[0, 'is_genderless'] + 0)

1.0


### Q158 - Science and technology are making our lives healthier, easier, and more comfortable

In [393]:
df_panel = df_fin.dropna(
    subset=[
        'X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 
        'Q273', 'Q274', 'Q289', 
        'Q158_gpt', 'Q158', 'Q158_yandex'
    ]
).reset_index()

df_panel = df_panel[df_panel['Q158'].isin(range(1, 11))]
df_panel = df_panel[df_panel['Q158_yandex'].isin(range(1, 11))]
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])

Y_real = df_panel['Q158']
Y_gpt = df_panel['Q158_gpt']
Y_yandex = df_panel['Q158_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q158   R-squared:                        0.0082
Estimator:                   PanelOLS   R-squared (Between):             -0.1169
No. Observations:               83315   R-squared (Within):               0.0082
Date:                Tue, Jun 10 2025   R-squared (Overall):             -0.0010
Time:                        18:01:45   Log-likelihood                -1.889e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      23.776
Entities:                          62   P-value                           0.0000
Avg Obs:                       1343.8   Distribution:                F(29,83224)
Min Obs:                       726.00                                           
Max Obs:                       3995.0   F-statistic (robust):             13.731
                            

### Q159 - Because of science and technology, there will be more opportunities for the next generation

In [396]:
df_panel = df_fin.dropna(
    subset=[
        'X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 
        'Q273', 'Q274', 'Q289', 
        'Q159_gpt', 'Q159', 'Q159_yandex'
    ]
).reset_index()

df_panel = df_panel[df_panel['Q159'].isin(range(1, 11))]
df_panel = df_panel[df_panel['Q159_yandex'].isin(range(1, 11))]
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])

Y_real = df_panel['Q159']
Y_gpt = df_panel['Q159_gpt']
Y_yandex = df_panel['Q159_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q159   R-squared:                        0.0049
Estimator:                   PanelOLS   R-squared (Between):             -0.0518
No. Observations:               83142   R-squared (Within):               0.0049
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0006
Time:                        18:08:04   Log-likelihood                -1.858e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      14.136
Entities:                          62   P-value                           0.0000
Avg Obs:                       1341.0   Distribution:                F(29,83051)
Min Obs:                       727.00                                           
Max Obs:                       3995.0   F-statistic (robust):             7.9627
                            

### Q160 - We depend too much on science and not enough on faith

In [394]:
df_panel = df_fin.dropna(
    subset=[
        'X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 
        'Q273', 'Q274', 'Q289', 
        'Q160_gpt', 'Q160', 'Q160_yandex'
    ]
).reset_index()

df_panel = df_panel[df_panel['Q160'].isin(range(1, 11))]
df_panel = df_panel[df_panel['Q160_yandex'].isin(range(1, 11))]
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])

Y_real = df_panel['Q160']
Y_gpt = df_panel['Q160_gpt']
Y_yandex = df_panel['Q160_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q160   R-squared:                        0.0108
Estimator:                   PanelOLS   R-squared (Between):              0.0448
No. Observations:               82409   R-squared (Within):               0.0108
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0152
Time:                        18:02:32   Log-likelihood                -2.008e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      31.075
Entities:                          62   P-value                           0.0000
Avg Obs:                       1329.2   Distribution:                F(29,82318)
Min Obs:                       724.00                                           
Max Obs:                       3995.0   F-statistic (robust):             8.1878
                            

### Q161 - One of the bad effects of science is that it breaks down people’s ideas of right and wrong

In [397]:
df_panel = df_fin.dropna(
    subset=[
        'X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 
        'Q273', 'Q274', 'Q289', 
        'Q161_gpt', 'Q161', 'Q161_yandex'
    ]
).reset_index()

df_panel = df_panel[df_panel['Q161'].isin(range(1, 11))]
df_panel = df_panel[df_panel['Q161_yandex'].isin(range(1, 11))]
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])

Y_real = df_panel['Q161']
Y_gpt = df_panel['Q161_gpt']
Y_yandex = df_panel['Q161_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q161   R-squared:                        0.0100
Estimator:                   PanelOLS   R-squared (Between):              0.1287
No. Observations:               81423   R-squared (Within):               0.0100
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0203
Time:                        18:39:50   Log-likelihood                -1.968e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      28.262
Entities:                          62   P-value                           0.0000
Avg Obs:                       1313.3   Distribution:                F(29,81332)
Min Obs:                       718.00                                           
Max Obs:                       3995.0   F-statistic (robust):             12.797
                            

### Q162 - It is not important for me to know about science in my daily life

In [398]:
df_panel = df_fin.dropna(
    subset=[
        'X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 
        'Q273', 'Q274', 'Q289', 
        'Q162_gpt', 'Q162', 'Q162_yandex'
    ]
).reset_index()

df_panel = df_panel[df_panel['Q162'].isin(range(1, 11))]
df_panel = df_panel[df_panel['Q162_yandex'].isin(range(1, 11))]
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])

Y_real = df_panel['Q162']
Y_gpt = df_panel['Q162_gpt']
Y_yandex = df_panel['Q162_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q162   R-squared:                        0.0217
Estimator:                   PanelOLS   R-squared (Between):              0.0286
No. Observations:               82586   R-squared (Within):               0.0217
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0223
Time:                        18:40:56   Log-likelihood                -2.013e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      63.191
Entities:                          62   P-value                           0.0000
Avg Obs:                       1332.0   Distribution:                F(29,82495)
Min Obs:                       607.00                                           
Max Obs:                       3995.0   F-statistic (robust):             13.810
                            

### Q177 - Claiming government benefits to which you are not entitled

In [399]:
df_panel = df_fin.dropna(
    subset=[
        'X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 
        'Q273', 'Q274', 'Q289', 
        'Q177_gpt', 'Q177'
    ]
).reset_index()

df_panel = df_panel[df_panel['Q177'].isin(range(1, 11))]
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])

Y_real = df_panel['Q177']
Y_gpt = df_panel['Q177_gpt']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q177   R-squared:                        0.0083
Estimator:                   PanelOLS   R-squared (Between):             -0.0434
No. Observations:               84285   R-squared (Within):               0.0083
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0023
Time:                        18:43:23   Log-likelihood                -1.974e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      24.287
Entities:                          63   P-value                           0.0000
Avg Obs:                       1337.9   Distribution:                F(29,84193)
Min Obs:                       727.00                                           
Max Obs:                       3997.0   F-statistic (robust):             9.6584
                            

### Q182 - Homosexuality

In [400]:
df_panel = df_fin.dropna(
    subset=[
        'X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 
        'Q273', 'Q274', 'Q289', 
        'Q182_gpt', 'Q182'
    ]
).reset_index()

df_panel = df_panel[df_panel['Q182'].isin(range(1, 11))]
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])

Y_real = df_panel['Q182']
Y_gpt = df_panel['Q182_gpt']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q182   R-squared:                        0.0541
Estimator:                   PanelOLS   R-squared (Between):              0.2269
No. Observations:               79165   R-squared (Within):               0.0541
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.1241
Time:                        18:46:31   Log-likelihood                -1.843e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      155.99
Entities:                          60   P-value                           0.0000
Avg Obs:                       1319.4   Distribution:                F(29,79076)
Min Obs:                       673.00                                           
Max Obs:                       3997.0   F-statistic (robust):             17.026
                            

### Q183 - Prostitution

In [405]:
df_panel = df_fin.dropna(subset=['is_genderless', 'Q183_yandex']).reset_index()
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])
df_panel = df_panel[df_panel['Q183'].isin(range(1, 11))]

Y_real = df_panel['Q183']
Y_gpt = df_panel['Q183_gpt']
Y_yandex = df_panel['Q183_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q183   R-squared:                        0.0292
Estimator:                   PanelOLS   R-squared (Between):              0.2054
No. Observations:               68914   R-squared (Within):               0.0292
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0619
Time:                        18:51:58   Log-likelihood                -1.569e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      68.992
Entities:                          57   P-value                           0.0000
Avg Obs:                       1209.0   Distribution:                F(30,68827)
Min Obs:                       4.0000                                           
Max Obs:                       3887.0   F-statistic (robust):             11.103
                            

#### ordinal logistic

In [ ]:
df_model = df_fin.copy()
df_model = df_fin.dropna(subset=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'Q183', 'Q183_yandex', 'Q183_gpt']).reset_index()
df_model = df_model[df_model['Q183'].isin(range(1, 11))]

X = pd.get_dummies(
    df_model[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289']],
    drop_first=True
)

y_real = df_model['Q183']
y_gpt = df_model['Q183_gpt']
y_yandex = df_model['Q183_yandex']

model_real = OrderedModel(y_real, X, distr='logit')
res_real = model_real.fit(method='bfgs')  # можно 'lbfgs', 'newton', и т.п.

model_gpt = OrderedModel(y_gpt, X, distr='logit')
res_gpt = model_gpt.fit(method='bfgs')

model_yandex = OrderedModel(y_yandex, X, distr='logit')
res_yandex = model_yandex.fit(method='bfgs')

print(res_real.summary())
print(res_gpt.summary())
print(res_yandex.summary())

Optimization terminated successfully.
         Current function value: 1.634560
         Iterations: 52
         Function evaluations: 55
         Gradient evaluations: 55
Optimization terminated successfully.
         Current function value: 1.156890
         Iterations: 39
         Function evaluations: 42
         Gradient evaluations: 42
Optimization terminated successfully.
         Current function value: 0.964364
         Iterations: 36
         Function evaluations: 38
         Gradient evaluations: 38
                             OrderedModel Results                             
Dep. Variable:                   Q183   Log-Likelihood:            -1.2682e+05
Model:                   OrderedModel   AIC:                         2.537e+05
Method:            Maximum Likelihood   BIC:                         2.538e+05
Date:                Tue, 10 Jun 2025                                         
Time:                        19:21:10                                         
No. Observ

### Q184 - Abortion

In [407]:
df_panel = df_fin.dropna(subset=['is_genderless', 'Q184_yandex']).reset_index()
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])
df_panel = df_panel[df_panel['Q184'].isin(range(1, 11))]

Y_real = df_panel['Q184']
Y_gpt = df_panel['Q184_gpt']
Y_yandex = df_panel['Q184_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q184   R-squared:                        0.0353
Estimator:                   PanelOLS   R-squared (Between):              0.2549
No. Observations:               75085   R-squared (Within):               0.0353
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0913
Time:                        18:52:56   Log-likelihood                -1.735e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      91.388
Entities:                          62   P-value                           0.0000
Avg Obs:                       1211.0   Distribution:                F(30,74993)
Min Obs:                       4.0000                                           
Max Obs:                       3887.0   F-statistic (robust):             12.683
                            

### Q185 - Divorce

In [408]:
df_panel = df_fin.dropna(subset=['is_genderless', 'Q185_yandex']).reset_index()
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])
df_panel = df_panel[df_panel['Q185'].isin(range(1, 11))]

Y_real = df_panel['Q185']
Y_gpt = df_panel['Q185_gpt']
Y_yandex = df_panel['Q185_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q185   R-squared:                        0.0334
Estimator:                   PanelOLS   R-squared (Between):              0.2610
No. Observations:               75329   R-squared (Within):               0.0334
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0925
Time:                        18:54:42   Log-likelihood                -1.815e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      86.532
Entities:                          62   P-value                           0.0000
Avg Obs:                       1215.0   Distribution:                F(30,75237)
Min Obs:                       4.0000                                           
Max Obs:                       3887.0   F-statistic (robust):             25.596
                            

#### Russia

In [ ]:
df_fin.dtype

In [443]:
df_rus = (
    df_fin[df_fin['code']=='USA']
    .dropna(subset=['is_genderless','Q185_yandex'])
    .reset_index(drop=True)
)

formula_real = (
    'Q185 ~ '
    'C(X003R2) + C(Q260) + C(Q275) + C(Q288R) + '
    'C(Q287) + C(Q273) + C(Q274) + C(Q289) + '
    'C(is_genderless)'
)

formula_yand = (
    'Q185_yandex ~ '
    'C(X003R2) + C(Q260) + C(Q275) + C(Q288R) + '
    'C(Q287) + C(Q273) + C(Q274) + C(Q289) + '
    'C(is_genderless)'
)

formula_gpt = (
    'Q185_gpt ~ '
    'C(X003R2) + C(Q260) + C(Q275) + C(Q288R) + '
    'C(Q287) + C(Q273) + C(Q274) + C(Q289) + '
    'C(is_genderless)'
)

# 3) Строим модели
res_real   = smf.ols(formula_real, data=df_rus).fit()
res_yandex = smf.ols(formula_yand,  data=df_rus).fit()
res_gpt = smf.ols(formula_gpt,  data=df_rus).fit()

# 4) Смотрим результаты
print(res_real.summary())
print(res_yandex.summary())
print(res_gpt.summary())

                            OLS Regression Results                            
Dep. Variable:                   Q185   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     10.22
Date:                Tue, 10 Jun 2025   Prob (F-statistic):           2.18e-44
Time:                        20:15:30   Log-Likelihood:                -5492.2
No. Observations:                2430   AIC:                         1.105e+04
Df Residuals:                    2399   BIC:                         1.123e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

### Q186 - Sex before marriage

In [409]:
df_panel = df_fin.dropna(subset=['is_genderless', 'Q186_yandex']).reset_index()
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])
df_panel = df_panel[df_panel['Q186'].isin(range(1, 11))]

Y_real = df_panel['Q186']
Y_gpt = df_panel['Q186_gpt']
Y_yandex = df_panel['Q186_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q186   R-squared:                        0.0543
Estimator:                   PanelOLS   R-squared (Between):              0.3308
No. Observations:               72666   R-squared (Within):               0.0543
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.1699
Time:                        18:56:00   Log-likelihood                -1.711e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      138.97
Entities:                          60   P-value                           0.0000
Avg Obs:                       1211.1   Distribution:                F(30,72576)
Min Obs:                       4.0000                                           
Max Obs:                       3887.0   F-statistic (robust):             19.230
                            

### Q187 - Suicide

In [410]:
df_panel = df_fin.dropna(subset=['is_genderless', 'Q187_yandex']).reset_index()
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])
df_panel = df_panel[df_panel['Q187'].isin(range(1, 11))]

Y_real = df_panel['Q187']
Y_gpt = df_panel['Q187_gpt']
Y_yandex = df_panel['Q187_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q187   R-squared:                        0.0237
Estimator:                   PanelOLS   R-squared (Between):              0.1838
No. Observations:               75148   R-squared (Within):               0.0237
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0471
Time:                        19:01:04   Log-likelihood                -1.643e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      60.776
Entities:                          62   P-value                           0.0000
Avg Obs:                       1212.1   Distribution:                F(30,75056)
Min Obs:                       4.0000                                           
Max Obs:                       3887.0   F-statistic (robust):             43.442
                            

### Q189 - For a man to beat his wife

In [403]:
df_panel = df_fin.dropna(subset=['is_genderless']).reset_index()
df_panel = df_panel.set_index(['country', df_panel.groupby('country').cumcount()])

Y_real = df_panel['Q189']
Y_gpt = df_panel['Q189_gpt']
Y_yandex = df_panel['Q189_yandex']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'is_genderless'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y_real, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

mod_gpt = PanelOLS(Y_gpt, X, entity_effects=True)
res_gpt = mod_gpt.fit(cov_type='clustered', cluster_entity=True)

mod_yandex = PanelOLS(Y_yandex, X, entity_effects=True)
res_yandex = mod_yandex.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)
print(res_gpt.summary)
print(res_yandex.summary)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   Q189   R-squared:                        0.0090
Estimator:                   PanelOLS   R-squared (Between):              0.1073
No. Observations:               76416   R-squared (Within):               0.0090
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.0170
Time:                        18:49:56   Log-likelihood                -1.544e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      23.043
Entities:                          62   P-value                           0.0000
Avg Obs:                       1232.5   Distribution:                F(30,76324)
Min Obs:                       4.0000                                           
Max Obs:                       3894.0   F-statistic (robust):             14.914
                            

In [383]:
df_panel = df_fin.set_index(['country', df.groupby('country').cumcount()])

Y = df_panel['Q182_gpt']

X = pd.get_dummies(
    df_panel[['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289']],
    columns=['X003R2', 'Q260', 'Q275', 'Q288R', 'Q287', 'Q273', 'Q274', 'Q289'],
    drop_first=True      # отбрасываем первую категорию каждой переменной
)
X = X.assign(const=1)

# 4. Строим панельную FE-модель с entity_effects (фиксы по странам):
mod = PanelOLS(Y, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)

print(res.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:               Q182_gpt   R-squared:                        0.4297
Estimator:                   PanelOLS   R-squared (Between):              0.7484
No. Observations:               85308   R-squared (Within):               0.4297
Date:                Tue, Jun 10 2025   R-squared (Overall):              0.6017
Time:                        17:55:34   Log-likelihood                -1.052e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2214.2
Entities:                          63   P-value                           0.0000
Avg Obs:                       1354.1   Distribution:                F(29,85216)
Min Obs:                       742.00                                           
Max Obs:                       3997.0   F-statistic (robust):             292.90
                            

## Регрессия mixed effects 

In [384]:
# X003R2                 # AGE: 1..3
# Q260                    # GENDER: 1 или 2
# Q275                # EDUCATION: 0..8
# Q288R                # INCOME: 1..3
# Q287        # SOCIAL CLASS: 1..5
# Q273        # MARITAL STATUS: 1..6
# Q274        # HOW MANY CHILDREN: 0..3
# Q289       # RELIGIOUS: 0..9

In [385]:
df_fin.head(1)

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q195_yandex,GDP_per_capita_2024,ict_idx,GDP_wvs_year,rd_idx_gpd,TradAgg,SurvSAgg,CPI_idx,HDI_idx,is_genderless
0,2018,AND,Andorra,3,2,1,2,3,1270,1,...,3.0,46359.0,88.8,42819.77458,NaN,0.5486,2.1823,NaN,0.873,1.0


In [386]:
df_fin_copy = df_fin[['Q182_gpt', 'X003R2', 'Q260', 'Q275' ,'Q288R', 'Q287', 'Q273', 'Q274', 'Q289', 'TradAgg', 'HDI_idx']].dropna()

In [387]:
# model_gpt_182 = smf.mixedlm("Q182_gpt ~ C(X003R2) + C(Q260) + C(Q275) + C(Q288R) + C(Q287) + C(Q273) + C(Q274) + C(Q289) + TradAgg + HDI_idx", data=df_fin_copy, groups="code")
# model_gpt_182.fit()